<a href="https://colab.research.google.com/github/nkatara/iiit-ai-ml/blob/main/Demo_Linear_Regression_Diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint
## Not for grading

## Learning Objective

At the end of the experiment, you will be able to understand:

*  how to implement linear regression for multiple variables using Scikit-Learn

## Dataset

#### Description
This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

The datasets consists of several medical predictor variables and one target variable, Outcome.

* Preg: Number of times pregnant
* Glucose: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
* BloodPressure: Diastolic blood pressure (mm Hg)
* SkinThickness: Triceps skin fold thickness (mm)
* Insulin: 2-Hour serum insulin (mu U/ml)
* BMI: Body mass index (weight in kg/(height in m)^2)
* DiabetesPedigreeFunction: Diabetes pedigree function
* Age: Age (years)
* Outcome: Class variable (0 or 1)

## Setup Steps

In [1]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "2501952" #@param {type:"string"}


In [2]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "8439328088" #@param {type:"string"}


In [3]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()

notebook= "Demo_Linear_Regression_Diabetes" #name of the notebook
Answer = "Ungraded"
def setup():
#  ipython.magic("sx pip3 install torch")
    from IPython.display import HTML, display
    ipython.magic("sx wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/diabetes.csv")
    display(HTML('<script src="https://dashboard.talentsprint.com/submissions/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():

    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword(), "batch" : ""}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getComplexity() and getAdditional() and getConcepts() and getComments():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "id" : Id, "file_hash" : file_hash,
              "feedback_experiments_input" : Comments, "notebook" : notebook, "batch" : ""}

      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://learn-iiith.talentsprint.com/notebook_submissions")
        # print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
      return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None
def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None

def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()

else:
  print ("Please complete Id and Password cells before running setup")


Setup completed successfully


## Import required packages

In [4]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

## Load the dataset

In [5]:
# Load the diabetes dataset
diabetes = pd.read_csv("diabetes.csv")

diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
diabetes_X = diabetes[["BloodPressure", "Age"]]  # Select specific columns using iloc is diabetes.iloc[:,[2, 7]]
diabetes_y = diabetes["Glucose"]                # Select glucose column using iloc is diabetes.iloc[:, 1]

print(diabetes_X.shape, diabetes_y.shape)

(768, 2) (768,)


To learn the linear-regression model from the training data, and predict the values for the test data, we will  perform the train-test split.

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(diabetes_X, diabetes_y, test_size=0.2)

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 2), (154, 2), (614,), (154,))

There are a few ways to find the best fit line. One of the approaches is the Ordinary Least Squares (OLS) method which is an intutive mathematical method.

**Note:** Refer the following [link](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) for Linear Regression from sklearn


In [21]:
# Create a linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
pred_train = regr.predict(X_train)
pred_test = regr.predict(X_test)
#diabetes_y_pred = regr.predict(X_test)

In [23]:
# Intercept : 85.3609047556678
# slop : 0.13781713
# variable1 : 25
#vaeiable 2: 52
#Slop:  0.76433925

#formula slop1 * var1 + slop2 * var2 + intercept
0.76433925 * 52 + 0.13781713 * 25 + 85.3609047556678

128.5519740056678

In [22]:
# For retrieving the slope use 'regr.coef_'. As we have taken two variables, we will get two coefficients m1 and m2
print('Coefficients: ', regr.coef_)

# To retrieve the intercept
print('Intercept: ', regr.intercept_)

# Calculate the root mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_train, pred_train))
print("Mean squared error: %.2f" % mean_squared_error(y_test, pred_test))
print("Root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, diabetes_y_pred)))

Coefficients:  [0.13781713 0.76433925]
Intercept:  85.3609047556678
Mean squared error: 883.61
Mean squared error: 1181.25
Root mean squared error: 34.37


In [15]:
print(y_train)

412    143
198    109
64     114
514     99
298    100
      ... 
272    122
558    103
727    141
331     87
678    121
Name: Glucose, Length: 614, dtype: int64


In [16]:
print(pred_train)

[113.7530067  114.05402117 126.55908335 111.14717143 131.27024583
 125.36860262 125.7947441  111.0592277  111.63587642 105.99806438
 121.63485012 123.28865562 109.68105644 125.8944909  142.39713675
 113.62787969 122.24868213 118.21391516 124.17812189 126.07037835
 111.36024217 113.01404768 112.52534268 111.20973493 113.62787969
 110.16976144 111.27229844 135.80640791 149.40131698 109.40542219
 112.43739896 117.93828091 112.92610395 113.41480894 135.91884481
 115.70782668 109.89412719 114.88092393 111.0592277  150.16565622
 112.52534268 110.16976144 125.00502464 110.99666419 118.15135165
 120.50693289 111.76100344 115.36962892 111.82356695 140.59282401
 113.13917469 111.54793269 113.01404768 114.11658468 118.48954941
 128.5257064  130.0172016  115.2445019  111.14717143 130.65641382
 134.54067356 132.83610762 139.86566806 114.66785318 128.06238162
 115.21912169 125.15553187 117.8757174  116.25909518 116.13396817
 110.23232495 116.62267316 147.59700423 144.84066172 114.05402117
 128.06238

In [24]:
from sklearn.metrics import r2_score
print(r2_score(y_train, pred_train))
print(r2_score(y_test, pred_test))

0.1004861154975949
-0.014291216939015694


In [12]:
print(regr.score(X_test, y_test))

-0.014291216939015694


### Please answer the questions below to complete the experiment:




In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Experiment walkthrough video? { run: "auto", vertical-output: true, display-mode: "form" }
Walkthrough = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook for Ungrading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")